In [29]:
abstract record Measure(string Unit);
record DiscreteMeasure(string Unit, uint Value) : Measure(Unit);
record ContinuousMeasure(string Unit, decimal Value) : Measure(Unit);

In [30]:
static Measure AsDiscriminatedUnion(this Measure m) =>
    m switch
    {
        DiscreteMeasure or ContinuousMeasure => m,
        _ => throw new ArgumentException("Measure must be either DiscreteMeasure or ContinuousMeasure")
    };

static TResult MapAny<TResult>(this Measure m,
    Func<DiscreteMeasure, TResult> discrete,
    Func<ContinuousMeasure, TResult> continuous) =>
    m.AsDiscriminatedUnion() switch
    {
        DiscreteMeasure d => discrete(d),
        ContinuousMeasure c => continuous(c),
        _ => default!
    };

static (Measure a, Measure b) SplitInHalves(this Measure m) => m.MapAny(
    d => (d with { Value = (d.Value + 1) / 2 }, d with { Value = d.Value / 2 }),
    c =>
    {
        Measure half = c with { Value = c.Value / 2 };
        return (half, half);
    });

In [31]:
new DiscreteMeasure("m", 5).SplitInHalves()

(DiscreteMeasure { Unit = m, Value = 3 }, DiscreteMeasure { Unit = m, Value = 2 }) Item1 DiscreteMeasure { Unit = m, Value = 3 } Value 3 Unit m Item2 DiscreteMeasure { Unit = m, Value = 2 } Value 2 Unit m

In [32]:
new ContinuousMeasure("m", 5.5m).SplitInHalves()

(ContinuousMeasure { Unit = m, Value = 2.75 }, ContinuousMeasure { Unit = m, Value = 2.75 }) Item1 ContinuousMeasure { Unit = m, Value = 2.75 } Value 2.75 Unit m Item2 ContinuousMeasure { Unit = m, Value = 2.75 } Value 2.75 Unit m